In [1]:
import ee
import datetime
import os
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

# append the parent path into sys-path so wen can import necessary modules
sys.path.append('../')

In [3]:
from BackGround_modules.Class_1_Make_fourier_imgs import Make_Fourier
from BackGround_modules.Class_2_Classify_Fourier_Img import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

The default supplementary data a Zero img, change accordingly when classify other years imgs



### Step_0_Define_Basic_Parameters

In [4]:
# imprt North_China_Plain boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [5]:
# Define the time-range
# just get one_range from every different sensors
year_range = [(f'{i}-01-01',f'2019-12-31') for i in range(2015,2020)] + \
             [(f'{i}-01-01',f'2013-12-31') for i in range(2009,2014)] + \
             [(f'{i}-01-01',f'2010-12-31') for i in range(2006,2011)]


In [6]:
year_range

[('2015-01-01', '2019-12-31'),
 ('2016-01-01', '2019-12-31'),
 ('2017-01-01', '2019-12-31'),
 ('2018-01-01', '2019-12-31'),
 ('2019-01-01', '2019-12-31'),
 ('2009-01-01', '2013-12-31'),
 ('2010-01-01', '2013-12-31'),
 ('2011-01-01', '2013-12-31'),
 ('2012-01-01', '2013-12-31'),
 ('2013-01-01', '2013-12-31'),
 ('2006-01-01', '2010-12-31'),
 ('2007-01-01', '2010-12-31'),
 ('2008-01-01', '2010-12-31'),
 ('2009-01-01', '2010-12-31'),
 ('2010-01-01', '2010-12-31')]

In [12]:
random_point = ee.FeatureCollection.randomPoints(North_China_Plain, 100, 101).toList(100).getInfo()

In [48]:
# randomly generated 1000 point within the boundary of North China Plain
# which will be used to record the residul between original and fitted
# normalized value
                                                                 

point_with_residule = []


# loop through all year_range
for span in year_range:
    
    # define the start and end of time
    start = span[0]
    end   = span[1]
    spane_len = str(int(end[:4]) - int(start[:4]))
    
    # get span_name for export
    split = '-'
    span_name = f'{start.split(split)[0]}_{end.split(split)[0]}'
    
    # get the folder to store computation result
    folder = 'Step_2_result'
    

    # using while-True to avoid internet-unstable caused break
           
    # loop through harmonic of 1-10
    for harmonic in range(1,11):

        # make a residule img 
        Fourier_making = Make_Fourier(start_date=start,end_date=end,harmonics=harmonic)
        Fourier_making.Stp_1_Create_hamonic_names()
        Fourier_making.Stp_2_Add_harmonics()
        Fourier_making.Stp_3_Harmonic_fit()

        # get the Residule img.
        Residule_img = ee.ImageCollection(Fourier_making.harmonicTrendResidule)

        # loop through all random point to extract residule value to ramdom point
        # here not use sample point is because 'memory exceed' problem

        pt_flag = None
        
        
        for subset_pt in [range(i,i+5) for i in range(0,100,5) ]:
            
            # get the first and last value of subset
            start = subset_pt[0]
            end   = subset_pt[-1]
            
            # get the subset points
            pt = random_point[start:end+1]
            
            # calculate the 
            point_with_residule.append(
                                      ee.FeatureCollection(pt)\
                                        .map(lambda p: Residule_img.getRegion(ee.FeatureCollection(p)
                                                                                .geometry(),30))
                                                                                .getInfo())

            #print out the process out point calculation
            print(f'{span_name}_harmonic_{harmonic}_pt_{str(start).zfill(2)}_{str(end).zfill(2)}')

        # print out the process of Harmonic
        print(f'_____Harmonic_{harmonic}____')

        
    # print out the process of year_span
    print(f'===============|{span_name}|==================')

Analyzing the images of 2015_2019


EEException: Collection.map: A mapped algorithm must return a Feature or Image.

In [35]:
list(subset_pt)

[0, 1, 2, 3, 4]

In [44]:
random_point[start:end]

[{'type': 'Feature',
  'geometry': {'type': 'Point',
   'coordinates': [118.0645505058396, 36.33837078836222]},
  'id': '0',
  'properties': {}},
 {'type': 'Feature',
  'geometry': {'type': 'Point',
   'coordinates': [117.45136726995973, 34.74709523190146]},
  'id': '1',
  'properties': {}},
 {'type': 'Feature',
  'geometry': {'type': 'Point',
   'coordinates': [116.37995553940033, 40.93179452566714]},
  'id': '2',
  'properties': {}},
 {'type': 'Feature',
  'geometry': {'type': 'Point',
   'coordinates': [112.39193093143045, 34.22311157858832]},
  'id': '3',
  'properties': {}}]

### Step_1_Generate Fourier img with different harmonic from 1997-2019

In [ ]:
# randomly generated 1000 point within the boundary of North China Plain
# which will be used to record the residul between original and fitted
# normalized value
                                                                 
random_point = ee.FeatureCollection.randomPoints(North_China_Plain, 100, 101).toList(100).getInfo()
    
# loop through all year_range
for span in year_range:
    
    # define the start and end of time
    start = span[0]
    end   = span[1]
    spane_len = str(int(end[:4]) - int(start[:4]))
    
    # get span_name for export
    split = '-'
    span_name = f'{start.split(split)[0]}_{end.split(split)[0]}'
    
    # get the folder to store computation result
    folder = 'Step_2_result'
    
    while True:
        
        # using while-True to avoid internet-unstable caused break
        try:            
            # loop through harmonic of 1-10
            for harmonic in range(1,11):

                # make a residule img 
                Fourier_making = Make_Fourier(start_date=start,end_date=end,harmonics=harmonic)
                Fourier_making.Stp_1_Create_hamonic_names()
                Fourier_making.Stp_2_Add_harmonics()
                Fourier_making.Stp_3_Harmonic_fit()

                # get the Residule img.
                Residule_img = ee.ImageCollection(Fourier_making.harmonicTrendResidule)

                # loop through all random point to extract residule value to ramdom point
                # here not use sample point is because 'memory exceed' problem
        
                pt_flag = None
                for pt in random_point:
            
                    pt_flag = pt['id']
                    point_with_residule = Residule_img.getRegion(pt['geometry'],30).getInfo()
                    
                    #print out the process out point calculation
                    print(f'{span_name}_harmonic_{harmonic}_pt_{str(pt_flag).zfill(2)}')
                
                # print out the process of Harmonic
                print(f'_____Harmonic_{harmonic}____')
            
        
        except:
            
            print(f'Residule computation on {span_name}_harmonic_{harmonic}_pt_{pt_flag} not successful!')
            
            # if something wrong, do it again!
            continue
        
        break
        
    # print out the process of year_span
    print(f'===============|{span_name}|==================')

Analyzing the images of 2015_2019
Residule computation on 2015_2019_harmonic_1_pt_0 not successful!
Analyzing the images of 2015_2019
Residule computation on 2015_2019_harmonic_1_pt_0 not successful!
Analyzing the images of 2015_2019
Residule computation on 2015_2019_harmonic_1_pt_0 not successful!
Analyzing the images of 2015_2019
Residule computation on 2015_2019_harmonic_1_pt_0 not successful!
Analyzing the images of 2015_2019
Residule computation on 2015_2019_harmonic_1_pt_0 not successful!
Analyzing the images of 2015_2019
Residule computation on 2015_2019_harmonic_1_pt_0 not successful!
Analyzing the images of 2015_2019
Residule computation on 2015_2019_harmonic_1_pt_0 not successful!
Analyzing the images of 2015_2019
Residule computation on 2015_2019_harmonic_1_pt_0 not successful!
Analyzing the images of 2015_2019
Residule computation on 2015_2019_harmonic_1_pt_0 not successful!
Analyzing the images of 2015_2019
Residule computation on 2015_2019_harmonic_1_pt_0 not successful!
